# Theorische vragen Deep Learning

Sabine De Vreese, Stijn Lievens en Simon De Gheselle

24 oktober 2025

## Inleiding
Hieronder vind je een vragenlijst die je kan helpen bij het studeren voor het theorie-examen van het opleidingsonderdeel Deep Learning. Veel vragen op het theorie-examen zullen gebaseerd zijn op de lijst van onderstaande vragen. Het is dus een goed idee als je minstens deze vragen kan beant-woorden op het moment dat je het examen aflegt.

## Hoofdstuk 13: Inladen en Voorbehandelen van Data met Tensorflow
1. Deze vraag gaat over “embeddings”.
- Wat is een “embedding”?
    > Embedding zet een token om naar een vector van getallen, waarbij positie in ruimte betekenis van het woordt geeft.

- Wat zijn de hyperparameters van een “embedding” laag?
    > Vocabulary size (woordenschat grootte), bv. [UNK, 'Tafel', 'Stoel']\
    > Embedding dim. (woord grootte)\
    > Input length (seq length), korte zinnen worden opgevuld met *padding*
- Hoeveel trainbare parameters bevat een “embedding” laag?
    > Elk woord krijgt een willekeurige vector. Waarbij het model meer tekst ziet, worden de getallen van de vector aangepast
    >
    > $\text{Aantal parameters} = \text{Vocabulaire grootte} \times \text{Embedding dimensie}$
    >
    > $10\;000\;(\text{woorden}) * 256\;(\text{dimensie}) = 2\;560\;000\;\text{parameters}$

2. Beschrijf in detail hoe de `shuffle` methode van `tf.data.Dataset` werkt. Wat is het effect van de parameter `buffer_size`?
    > Shuffle schudt elementen van de dataset door middel van een buffer. Eén element komt de buffer binnen op basis van de buffer_size, en één willekeurig element wordt geretourneerd (herhaald voor elk element). Dit is efficiënt voor RAM-geheugen.

In [24]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3,4,5,6,7,8,9,10])
dataset = dataset.shuffle(buffer_size=4)
for element in dataset:
    print(element)

tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


2025-12-28 15:01:40.243459: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


3. Deze vraag gaat over de methode [TF-IDF](https://www.youtube.com/watch?v=vZAXpvHhQow).
- Wat betekent de afkorting TF-IDF?
    > Term Frequency - Inverse Document Frequency
- Wat zijn de twee factoren die de TF-IDF score bepalen van een woord in een document dat zelf behoort tot een corpus van documenten. Het is voldoende om dit op een kwalitatieve manier te beschrijven. (Je moet m.a.w. geen precieze formules kunnen geven.)
    > Aantal keer het woord voorkomt in een document (TF) en aantal keer het woord voorkomt in de documenten van het corups.
- Is TF-IDF een ijle (“sparse”) of een dichte (“dense”) representatie van een document?
    > Ja **sparse**, want alles wordt op `nullen` gezet, behalve het gezochte woord niet.
- Wat is het voordeel van het gebruik van TF-IDF vergeleken met het eenvoudig tellen van woordvoorkomens?
    > Met TF-IDF kun je zien hoe belangrijk een woord is, ten opzichte van gewoon tellen. Woorden die veel voorkomen zoals `stopwoorden` worden genegeerd.

    Voorbeeld:

    Corups D\
        $d_1$ A quick brown *fox* jumps over the lazy dogs. What a *fox*!\
        $d_2$ A quick brown *fox* jumps over the lazy *fox*. What a *fox*!
    
    Term-frequency
    - $TF("fox,d1") = \frac{2\;\text{aantal keer `fox`}}{12\;\text{aantal woorden in zin}} = 0.16$\
    - $TF("fox,d2") = \frac{3\;\text{aantal keer `fox`}}{12\;\text{aantal woorden in zin}}= 0.25$

    Inverse document frequency
    - $IDF("fox",D) = \log(\frac{2\;\text{aantal documenten}}{2\;\text{aantal documenten woord voorkomt}}) = 0$
    
    TF-IDF Score
    - $\text{TF-IDF}("fox",d_1,D) = 0.16 * 0 = 0$
    - $\text{TF-IDF}("fox",d_2,D) = 0.25 * 0 = 0$

    $\rightarrow$ Het woordt `fox` is een algemeen woord, omdat het in elk document staat. Dit betekent dat het niet zo belangrijk is.

4. Voor dit hoofdstuk moet je ook in staat zijn om eenvoudige vragen i.v.m.de verschillende “preprocessing” lagen in Keras te beantwoorden. Enkelevoorbeeldvragen worden hieronder gegeven.

-   Wat is de uitvoer van onderstaande code?


In [42]:
import keras as ks

age = ks.ops.convert_to_tensor([
    [10.], [88.], [77.], [16.], [45.], [25.]
])
discretize_layer = ks.layers.Discretization(bin_boundaries=[18., 35., 65.])
# * 0 < 18 = [10,16] -> Bin 0
# * 18 <= x 35 = [25] -> Bin 1
# * 35 <= X < 65 = [45] -> Bin 2
# * x >= 65 = [77, 88] -> Bin 3
age_categories = discretize_layer(age)
print("--- age_categories -> discretize_layer(age) ---")
print(age_categories)
# * [[0], [3], [3], [0], [2], [1]]

onehot_layer = ks.layers.CategoryEncoding(num_tokens=4)
print("--- onehot_layer -> onehot_layer(age_categories) ---")
print(onehot_layer(age_categories))
#* [[1,0,0,0], [0,0,0,1], [0,0,0,1], [1,0,0,0],[0,0,1,0], [0,1,0,0]]

print("------")
two_age_categories = ks.ops.convert_to_tensor([[3, 1], [3, 0], [2, 1]])
print("--- two_age_categories -> onehot_layer(two_age_categories) ---")
print(onehot_layer(two_age_categories))
#* [[0,1,0,1], [1,0,0,1],[0,1,1,0]]

onehot_layer = ks.layers.CategoryEncoding(num_tokens=4 + 4)
print("--- onehot_layer -> onehot_layer(two_age_categories) ---")
print(onehot_layer(two_age_categories + [0, 4])) #* [[3, 5], [3, 4], [2, 5]]
#* [[0,0,0,1,0,1,0,0], [0,0,0,1,1,0,0,0], [0,0,1,0,0,1,0,0]]

--- age_categories -> discretize_layer(age) ---
tf.Tensor(
[[0]
 [3]
 [3]
 [0]
 [2]
 [1]], shape=(6, 1), dtype=int32)
--- onehot_layer -> onehot_layer(age_categories) ---
tf.Tensor(
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]], shape=(6, 4), dtype=float32)
------
--- two_age_categories -> onehot_layer(two_age_categories) ---
tf.Tensor(
[[0. 1. 0. 1.]
 [1. 0. 0. 1.]
 [0. 1. 1. 0.]], shape=(3, 4), dtype=float32)
--- onehot_layer -> onehot_layer(two_age_categories) ---
tf.Tensor(
[[0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0.]], shape=(3, 8), dtype=float32)


-   De onderstaande code:
    Uitvoer
    ```python
    tf.Tensor(
        [[2]
        [4]
        [3]
        [0]], shape=(4, 1), dtype=int64)
    ```

`StringLookUp` zoekt worden in een lijst.
- index 0 $\rightarrow$ Out-Of-Vocabulary (OOV) / UNK, niet in de lijst stond.

In [66]:
cities = ["Brussel","Aalst","Gent","Kortrijk","Gent"]
str_lookup_layer = ks.layers.StringLookup()
str_lookup_layer.adapt(cities)
print(str_lookup_layer([["Kortrijk"], ["Aalst"], ["Brussel"], ["Brugge"],["Gent"]]))
print(str_lookup_layer.get_vocabulary())
#* ['[UNK]', 'Gent', 'Kortrijk', 'Brussel', 'Aalst']
#* Kortrijk -> 2
#* Aalst -> 4
#* Brussel -> 3
#* Brugge -> OOV / [UNK] -> 0
#* Gent -> 1

tf.Tensor(
[[2]
 [4]
 [3]
 [0]
 [1]], shape=(5, 1), dtype=int64)
['[UNK]', 'Gent', 'Kortrijk', 'Brussel', 'Aalst']


Verklaar.
> #* ['[UNK]', 'Gent', 'Kortrijk', 'Brussel', 'Aalst']\
> 
> #* Kortrijk -> 2\
> #* Aalst -> 4\
> #* Brussel -> 3\
> #* Brugge -> `OOV` / `'[UNK]'` -> 0\
> #* Gent -> 1

Gegeven de resultaten van bovenstaande code, wat is de uitvoer vanonderstaande code?
> - Elk waarde + '[UNK]' wordt gesorteerd van groot naar klein met `[UNK]` als eerste waarde. Daarna worden de positie van het woord in een array aangeduid met 1

In [ ]:
cities = ["Brussel","Aalst","Gent","Kortrijk","Gent"]
str_lookup_layer = ks.layers.StringLookup(output_mode="one_hot")
str_lookup_layer.adapt(cities)
#* ['[UNK]', 'Gent', 'Kortrijk', 'Brussel', 'Aalst']
#* [[0 0 0 0 1], [0,0,1,0,0], [0,1,0,0,0], [0,0,0,0,1]]
print(str_lookup_layer([["Aalst"], ["Kortrijk"], ["Gent"], ["Aalst"]]))

tf.Tensor(
[[0 0 0 0 1]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 0 0 0 1]], shape=(4, 5), dtype=int64)


De onderstaande code, geeft als uitvoer
```python
<tf.Tensor: shape=(5, 1), dtype=int64, numpy=array([[5],[1],[1],[6],[7]])>
```
Verklaar
- `num_oov_indices=5` voegt 5 `[UNK]` voor aan de vocabulary lijst, daarom zijn de indices anders.

    `['[UNK]','[UNK]','[UNK]','[UNK]','[UNK]', 'Gent', 'Kortrijk', 'Brussel', 'Aalst']`


In [70]:
str_lookup_layer = ks.layers.StringLookup(num_oov_indices=5)
str_lookup_layer.adapt(cities)
print(str_lookup_layer.get_vocabulary())
#* ['[UNK]','[UNK]','[UNK]','[UNK]','[UNK]', 'Gent', 'Kortrijk', 'Brussel', 'Aalst']
#* [5], [1], [1], [6], [7]
print(str_lookup_layer([["Gent"], ["Hasselt"], ["Brugge"], ["Kortrijk"], ["Brussel"]]))

['[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'Gent', 'Kortrijk', 'Brussel', 'Aalst']
tf.Tensor(
[[5]
 [1]
 [1]
 [6]
 [7]], shape=(5, 1), dtype=int64)


De volgende code
geeft als uitvoer
```python
tf.Tensor(
    [[6 3 5 0 0]
    [1 7 3 5 4]], shape=(2, 5), dtype=int64)
```
- `TextVectorization` indices:
  - 0 -> '' -> padding
  - 1 -> '[UNK]' -> OOV

In [72]:
train_data = ["Amai dat weet ik niet",
              "Amai, amai, amai.",
              "Dat weet je niet",
              "Weet je dat niet? Amai"]
text_vec_layer = ks.layers.TextVectorization()
text_vec_layer.adapt(train_data)
print(text_vec_layer.get_vocabulary())
#* ['', '[UNK]', 'amai', 'weet', 'niet', 'dat', 'je', 'ik']
#* [6 3 5 0 0]
#* [1 7 3 5 4]
print(text_vec_layer(["Je weet dat","Tja, ik weet dat niet"]))

['', '[UNK]', 'amai', 'weet', 'niet', 'dat', 'je', 'ik']
tf.Tensor(
[[6 3 5 0 0]
 [1 7 3 5 4]], shape=(2, 5), dtype=int64)


Wat is de uitvoer van de onderstaande code?

```python
print(text_vec_layer(
    ["Weet zij dat? Amai",
    "Neen, ik wist dat"]
    ))
```
> $[[3 1 5 2]$\
> $[1 7 1 5]]$\
> shape(2, 4)

In [73]:
print(text_vec_layer(
    ["Weet zij dat? Amai",
    "Neen, ik wist dat"]
    ))

tf.Tensor(
[[3 1 5 2]
 [1 7 1 5]], shape=(2, 4), dtype=int64)
